### Calculate spatially correlated random error on a DoD

- Uses a shapefile polygon to define a stable area
- imports the DoD as a Geotiff raster
- Clips DoD raster using a polygon
- converts clipped raster to x, y, z points
- performs Kriging

In [85]:
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np
import pandas as pd
from pykrige.ok import OrdinaryKriging

from pyproj import CRS

from pathlib import Path

In [90]:
indod = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\DoD_2017-HDR_Reinterp_script_gen_clipped.tif")
inshp = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\shp\Stable_areas_script_zonal_stats.shp")

out_clip_ras = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\temp_clipped.tif")
out_clip_pts_shp = Path(r"C:\Users\jlogan\OneDrive - DOI\LRDP\LosPadresRes\DoD\temp_clipped_pts.shp")

stable_poly_id = [1, 2]

In [91]:
# Read the polygon shapefile using geopandas
polygon_gdf = gpd.read_file(inshp)

#filter only stable polygons
polygon_gdf = polygon_gdf.loc[polygon_gdf['id'].isin(stable_poly_id)]

# Read the raster using rasterio
with rasterio.open(indod) as src:
    # Clip the raster using the shapefile geometry
    clipped_raster, clipped_transform = rasterio.mask.mask(src, polygon_gdf.geometry, crop=True)
    clipped_meta = src.meta

# Update the metadata of the clipped raster
clipped_meta.update({
    'height': clipped_raster.shape[1],
    'width': clipped_raster.shape[2],
    'transform': clipped_transform
})

# # Write the clipped raster to the output path
# with rasterio.open(out_clip_ras, 'w', **clipped_meta) as dst:
#     dst.write(clipped_raster)

# Extract the x, y coordinates from the transform
x_coords = []
y_coords = []
z_values = []

for row in range(clipped_raster.shape[1]):
    for col in range(clipped_raster.shape[2]):
        x, y = rasterio.transform.xy(clipped_transform, row, col)
        value = clipped_raster[0, row, col]  # Assume single-band raster

        if value != clipped_meta['nodata']:
            x_coords.append(x)
            y_coords.append(y)
            z_values.append(value)

# Create a pandas DataFrame with x, y, z values
df = pd.DataFrame({'x': x_coords, 'y': y_coords, 'z': z_values})

#output points as shape
geometry = gpd.points_from_xy(df['x'], df['y'])
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.crs = clipped_meta['crs']
gdf.to_file(out_clip_pts_shp, driver='ESRI Shapefile')
    
# Define the kriging model without nugget effect
OK = OrdinaryKriging(x_coords[:100], y_coords[:100], z_values[:100], variogram_model='spherical')

# Perform ordinary kriging
z_pred, sigmasq_pred = OK.execute('grid', x_coords[:100], y_coords[:100])

# Get the sill value
sill = OK.variogram_model

# Get the range
range_value = OK.variogram_model

# Print the sill value and range
print(f'Using {OK.variogram_model} Variogram Model')
print(f'Partial Sill: {OK.variogram_model_parameters[0]}')
print(f'Full Sill: {OK.variogram_model_parameters[0] + OK.variogram_model_parameters[2]}')
print(f'Range: {OK.variogram_model_parameters[1]}')
print(f'Nugget: {OK.variogram_model_parameters[2]}')


Using spherical Variogram Model
Partial Sill: 0.004221612803161986
Full Sill: 0.004221612803162599
Range: 21.55845406779102
Nugget: 6.135545473223874e-16
